In [1]:
import csv
import math
from sympy import factorint

# Input and output file paths
input_path = 'coprime_pairs.csv'
output_path = 'coprime_pairs_sf.csv'

# Read CSV into list of integer pairs
with open(input_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    pairs = [tuple(map(int, row)) for row in reader]

# Filter by: coprime + square-free product
filtered = []
for a, b in pairs:
    if math.gcd(a, b) == 1:
        product = a * b
        factors = factorint(product)
        if all(exp == 1 for exp in factors.values()):
            filtered.append((a, b))

# Write result to output CSV
with open(output_path, mode='w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerows(filtered)


In [7]:
import numpy as np
import random
from typing import Set, List
import time

class NegativeDefiniteMatrixGenerator:
    """
    Efficiently generates unique negative definite integer matrices 
    with specified weight constraints for H-shaped graph adjacency matrices.
    """
    
    def __init__(self):
        self.generated_matrices: Set[bytes] = set()
        self.matrices: List[np.ndarray] = []
        
    def matrix_to_bytes(self, matrix: np.ndarray) -> bytes:
        """Convert matrix to bytes for duplicate detection."""
        return matrix.tobytes()
    
    def generate_h_adjacency_matrix(self, left_leg: int, right_leg: int, 
                                   bottom_left_leg: int, bottom_right_leg: int) -> np.ndarray:
        """Generate adjacency matrix for H-shaped graph with given leg lengths."""
        n = left_leg + right_leg + bottom_left_leg + bottom_right_leg + 4
        A = np.zeros((n, n), dtype=int)
        
        # Define node ranges
        top_left = list(range(left_leg + 1))
        top_right = list(range(left_leg + 1, left_leg + right_leg + 2))
        bottom_left = list(range(left_leg + right_leg + 2, left_leg + right_leg + bottom_left_leg + 3))
        bottom_right = list(range(left_leg + right_leg + bottom_left_leg + 3, n))
        
        # End nodes
        end_nodes = {top_left[0], top_right[-1], bottom_left[0], bottom_right[-1]}
        
        # Add edges with weights
        edges = []
        
        # Top legs
        for i in range(len(top_left) - 1):
            edges.append((top_left[i], top_left[i + 1]))
        for i in range(len(top_right) - 1):
            edges.append((top_right[i], top_right[i + 1]))
            
        # Bottom legs  
        for i in range(len(bottom_left) - 1):
            edges.append((bottom_left[i], bottom_left[i + 1]))
        for i in range(len(bottom_right) - 1):
            edges.append((bottom_right[i], bottom_right[i + 1]))
            
        # H-bar connections
        edges.extend([
            (top_left[-1], bottom_left[-1]),
            (top_right[0], bottom_right[0]),
            (top_left[-1], top_right[0])
        ])
        
        # Assign weights
        for i, j in edges:
            if i in end_nodes or j in end_nodes:
                weight = random.randint(-8, -1)  # < 0
            else:
                weight = random.randint(-10, -2)  # < -1
            A[i, j] = A[j, i] = weight
            
        # Diagonal entries
        for i in range(n):
            if i in end_nodes:
                A[i, i] = random.randint(-5, -1)
            else:
                A[i, i] = random.randint(-12, -2)
                
        return A
    
    def make_negative_definite(self, A: np.ndarray, max_attempts: int = 50) -> np.ndarray:
        """Force matrix to be negative definite while preserving structure."""
        original_A = A.copy()
        
        for attempt in range(max_attempts):
            try:
                eigenvals = np.linalg.eigvals(A)
                if np.all(eigenvals < -1e-10):
                    return A
                    
                # Make more negative definite
                min_eval = np.min(eigenvals)
                if min_eval >= 0:
                    shift = int(abs(min_eval)) + random.randint(2, 8)
                    np.fill_diagonal(A, A.diagonal() - shift)
                else:
                    # Strengthen negative definiteness
                    np.fill_diagonal(A, A.diagonal() - random.randint(1, 4))
                    
            except:
                # Reset and try different approach
                A = original_A.copy()
                np.fill_diagonal(A, A.diagonal() - random.randint(5, 15))
                
        return A
    
    def generate_single_matrix(self) -> np.ndarray:
        """Generate a single unique negative definite matrix."""
        max_attempts = 1000
        
        for _ in range(max_attempts):
            # Random H-graph configuration
            legs = [random.randint(1, 4) for _ in range(4)]
            A = self.generate_h_adjacency_matrix(*legs)
            A = self.make_negative_definite(A)
            
            # Check uniqueness
            matrix_bytes = self.matrix_to_bytes(A)
            if matrix_bytes not in self.generated_matrices:
                self.generated_matrices.add(matrix_bytes)
                return A
                
        # If we can't find unique matrix with H-structure, generate random negative definite
        return self.generate_random_negative_definite()
    
    def generate_random_negative_definite(self) -> np.ndarray:
        """Generate random negative definite matrix as fallback."""
        n = random.randint(8, 20)
        
        # Start with random symmetric matrix
        A = np.random.randint(-10, 0, (n, n))
        A = (A + A.T) // 2
        
        # Make strongly negative definite
        np.fill_diagonal(A, A.diagonal() - np.random.randint(10, 30, n))
        
        return A
    
    def generate_matrices(self, count: int = 10000) -> List[np.ndarray]:
        """Generate specified number of unique negative definite matrices."""
        print(f"Generating {count} unique negative definite matrices...")
        start_time = time.time()
        
        batch_size = 100
        progress_interval = 1000
        
        while len(self.matrices) < count:
            batch = []
            for _ in range(min(batch_size, count - len(self.matrices))):
                matrix = self.generate_single_matrix()
                batch.append(matrix)
            
            self.matrices.extend(batch)
            
            if len(self.matrices) % progress_interval == 0:
                elapsed = time.time() - start_time
                rate = len(self.matrices) / elapsed
                print(f"Generated {len(self.matrices)}/{count} matrices ({rate:.1f} matrices/sec)")
        
        elapsed = time.time() - start_time
        print(f"Completed! Generated {len(self.matrices)} unique matrices in {elapsed:.2f} seconds")
        print(f"Average rate: {len(self.matrices)/elapsed:.1f} matrices/second")
        
        return self.matrices
    
    def verify_matrices(self, sample_size: int = 100):
        """Verify properties of generated matrices."""
        if not self.matrices:
            print("No matrices to verify")
            return
            
        sample = random.sample(self.matrices, min(sample_size, len(self.matrices)))
        
        negative_definite_count = 0
        constraint_violations = 0
        
        print(f"\nVerifying {len(sample)} matrices...")
        
        for matrix in sample:
            # Check negative definiteness
            try:
                eigenvals = np.linalg.eigvals(matrix)
                if np.all(eigenvals < -1e-10):
                    negative_definite_count += 1
            except:
                pass
                
        print(f"Negative definite: {negative_definite_count}/{len(sample)} ({100*negative_definite_count/len(sample):.1f}%)")
        print(f"Unique matrices: {len(self.generated_matrices)}")
        print(f"Matrix sizes range: {min(m.shape[0] for m in sample)} to {max(m.shape[0] for m in sample)}")

def generate_10000_matrices():
    """Main function to generate 10,000 unique matrices."""
    generator = NegativeDefiniteMatrixGenerator()
    matrices = generator.generate_matrices(10000)
    generator.verify_matrices(200)
    return matrices

if __name__ == "__main__":
    # Set seed for reproducibility (remove for true randomness)
    random.seed(42)
    np.random.seed(42)
    
    # Generate the matrices
    matrices = generate_10000_matrices()
    
    # Show some examples
    print(f"\nExample matrices:")
    for i in range(3):
        print(f"\nMatrix {i+1} (shape {matrices[i].shape}):")
        print(matrices[i])
        eigenvals = np.linalg.eigvals(matrices[i])
        print(f"Min eigenvalue: {np.min(eigenvals):.3f}")

Generating 10000 unique negative definite matrices...
Generated 1000/10000 matrices (5512.7 matrices/sec)
Generated 2000/10000 matrices (5537.6 matrices/sec)
Generated 3000/10000 matrices (5087.2 matrices/sec)
Generated 4000/10000 matrices (4895.6 matrices/sec)
Generated 5000/10000 matrices (5052.5 matrices/sec)
Generated 6000/10000 matrices (5139.8 matrices/sec)
Generated 7000/10000 matrices (5217.8 matrices/sec)
Generated 8000/10000 matrices (5255.5 matrices/sec)
Generated 9000/10000 matrices (5287.7 matrices/sec)
Generated 10000/10000 matrices (5336.2 matrices/sec)
Completed! Generated 10000 unique matrices in 1.87 seconds
Average rate: 5336.2 matrices/second

Verifying 200 matrices...
Negative definite: 200/200 (100.0%)
Unique matrices: 10000
Matrix sizes range: 8 to 20

Example matrices:

Matrix 1 (shape (11, 11)):
[[-17  -5   0   0   0   0   0   0   0   0   0]
 [ -5 -17  -7   0   0   0   0 -10   0   0   0]
 [  0  -7 -16  -6   0   0   0   0  -9   0   0]
 [  0   0  -6 -18   0   0  

In [50]:
import csv

pairs = []
with open("pairs.csv", newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        pairs.append(list(map(int, row)))

In [2]:
import json
import math
import numpy as np
import os

# with open(f"data/seifert/projections/{pairs[i][0]}_{pairs[i][1]}a.json") as f:
#     data = json.load(f)
#     print(data)

def readpsi(m, r):
    path = f"data/theta/{m}_{r}.json"
    with open(path) as f:
        data = json.load(f)
    return data

def psi(m,r):
    return {
        'pref': r*r / (4 * m),
        'expo': [m * n*n for n in range(-50,51)],
        'signs': [-1] * 50 + [1] * 50
    }

def series(m, pos, coef):
    allsgn = np.array([])
    allexpo = np.array([])
    for p in range(len(pos)//2):
        data = readpsi(m,pos[p])
        if p == 0: overall = data['pref']
        allsgn = np.concat([allsgn,np.array(data['signs'])*coef[p]])
        allexpo = np.concat([allexpo, np.array(data['expo'])+int(data['pref']-overall)])
    lst = sorted(zip(allexpo, allsgn))
    allexpo, allsgn = zip(*lst)
    return {'factor': overall, 
            'expo': np.asarray(allexpo[:20]).astype(int).tolist(), 
            'sign': np.asarray(allsgn[:20]).astype(int).tolist() }
        

In [253]:
n = len(pairs)
for i in range(n):
    m = math.prod(pairs[i])
    with open(f"data/seifert/projections/{pairs[i][0]}_{pairs[i][1]}a.json") as f:
        data = json.load(f)
    for j in range(len(data['r'])):
        for k in range(len(data['pos'][j])//2):
            r = data['pos'][j][k]
            psi_data = psi(m, r)
            with open(f"data/theta/{m}_{r}.json", "w", encoding="utf-8") as f:
                json.dump(psi_data, f, indent=2, ensure_ascii=False)

In [270]:
mr = []
fac = []
expo = []
sgn = []
n = len(pairs)
for i in range(n):
    m = math.prod(pairs[i])
    with open(f"data/seifert/projections/{pairs[i][0]}_{pairs[i][1]}a.json") as f:
        data = json.load(f)
    for j in range(len(data['r'])):
        mr.append([m,data['r'][j]])
        tot = series(m, data['pos'][j], data['coef'][j])
        fac.append(tot['factor'])
        expo.append(tot['expo'])
        sgn.append(tot['sign'])

In [279]:
trc = []
for i in range(n):
    m = math.prod(pairs[i])
    with open(f"data/seifert/projections/{pairs[i][0]}_{pairs[i][1]}a.json") as f:
        data = json.load(f)
    for j in range(len(data['r'])):
        if len(data['pos'][j])==2:
            trc.append(2)
        else:
            cs = 0
            for k in range(2):
                cs += data['coef'][j][k]
            trc.append(cs)

In [287]:
with open("training_data/trc.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows([[x] for x in trc])


In [271]:
# save fac as a csv file where elements are floats
with open("training_data/fac.csv", "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    for x in fac:
        s = format(x, '.8g') if isinstance(x, float) else x
        w.writerow([s])    # one value per row
        
with open("training_data/mr.csv", "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerows(mr)

with open("training_data/expo.csv", "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerows(expo)

with open("training_data/sgn.csv", "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerows(sgn)


In [276]:
sgned = np.array(sgn) * np.array(expo)

In [277]:
with open("training_data/sgned_expo.csv", "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerows(sgned)


In [4]:
import csv
with open("training_data/sgned_expo.csv", "r", newline="", encoding="utf-8") as f:
    reader = csv.reader(f)
    sgned_expo = [list(map(int, row)) for row in reader]

In [8]:
sgned_expo4 = [x[:4] for x in sgned_expo]

In [9]:
sgned_expo4[0]

[0, -1, -6, 6]

In [10]:
with open("training_data/sgned_expo4.csv", "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerows(sgned_expo4)